<a href="https://colab.research.google.com/github/mbridgets/TheMUGSSProject3/blob/master/loading_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-2.4.6'
spark_version = 'spark-2.4.7'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [22]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [23]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://mugssproject3avodatageo.s3-us-west-1.amazonaws.com/avo_geo_data.csv"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("avo_geo_data.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
user_data_df.show()

+---+----------+-------------+------------+---------+---------+---------+----------+----------+----------+-----------+------------+----+------+-----+---------+----------+--------+---------+
|_c0|      date|average_price|total_volume|type_4046|type_4225|type_4770|total_bags|small_bags|large_bags|xlarge_bags|        type|year|  city|state|   region|city_state|latitude|longitude|
+---+----------+-------------+------------+---------+---------+---------+----------+----------+----------+-----------+------------+----+------+-----+---------+----------+--------+---------+
|  0|12/27/2015|         1.33|    64236.62|  1036.74| 54454.85|    48.16|   8696.87|   8603.62|     93.25|        0.0|conventional|2015|Albany|   NY|Northeast|Albany, NY| 42.6664| -73.7987|
|  1|12/20/2015|         1.35|    54876.98|   674.28| 44638.81|    58.33|   9505.56|   9408.07|     97.49|        0.0|conventional|2015|Albany|   NY|Northeast|Albany, NY| 42.6664| -73.7987|
|  2|12/13/2015|         0.93|   118220.22|    794

In [24]:
avo_geo_data = user_data_df.withColumnRenamed("_c0", "id")

In [32]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mugssproject3-awsdb.cd7njcklugjr.us-west-1.rds.amazonaws.com:5432/MUGSSProject3DB"
config = {"user":"MUGSSProject3DB", 
          "password": "data1234", 
          "driver":"org.postgresql.Driver"}


In [33]:
# Write DataFrame to active_user table in RDS

avo_geo_data.write.jdbc(url=jdbc_url, table='avocado_sales', mode=mode, properties=config)